# Assignment 1: Dino Fun World

You, in your role as a burgeoning data explorer and visualizer, have been asked by the administrators of a small amusement park in your hometown to answer a couple questions about their park operations. In order to perform the requested analysis, they have provided you with a database containing information about one day of the park's operations.

### Provided Database

The database provided by the park administration is formatted to be readable by any SQL database library. The course staff recommends the sqlite3 library. The database contains three tables, named 'checkins', 'attractions', and 'sequences'. The information contained in each of these tables is listed below:

`checkins`:
    - Description: check-in data for all visitors for the day in the park. The data includes two types of check-ins, inferred and actual checkins.
    - Fields: visitorID, timestamp, attraction, duration, type
`attraction`:
    - The attractions in the park by their corresponding AttractionID, Name, Region, Category, and type. Regions are from the VAST Challenge map such as Coaster Alley, Tundra Land, etc. Categories include Thrill rides, Kiddie Rides, etc. Type is broken into Outdoor Coaster, Other Ride, Carussel, etc.
    - Fields: AttractionID, Name, Region, Category, type
`sequences`:
    - The check-in sequences of visitors. These sequences list the position of each visitor to the park every five minutes. If the visitor has not entered the part yet, the sequence has a value of 0 for that time interval. If the visitor is in the park, the sequence lists the attraction they have most recently checked in to until they check in to a new one or leave the park.
    - Fields: visitorID, sequence
    
The database is named 'dinofunworld.db' and is located in the 'readonly' directory of the Jupyter Notebook environment. It can be accessed at 'readonly/dinofunworld.db'.

### Questions to Answer

The administrators would like you to answer four relatively simple questions about the park activities on the day in question. These questions all deal with park operations and can be answered using the data provided.

Question 1: What is the most popular attraction to visit in the park?
Question 2: What ride (note that not all attractions are rides) has the longest visit time?
Question 3: Which Fast Food offering has the fewest visitors?
Question 4: Compute the Skyline of number of visits and visit time for the park's ride and report the rides that appear in the Skyline.

#### Administrative Notes

This assignment will be graded by Coursera's grading system. In order for your answers to be correctly registered in the system, you must place the code for your answers in the cell indicated for each question. In addition, you should submit the assignment with the output of the code in the cell's display area. The display area should contain only your answer to the question with no extraneous information, or else the answer may not be picked up correctly. Each cell that is going to be graded has a set of comment lines at the beginning of the cell. These lines are extremely important and must not be modified or removed.

In [1]:
# Graded Cell, PartID: NDnou
# Question 1: What is the most popular attraction to visit in the park?
# Notes: Your output should be the name of the attraction.
# Graded Cell, PartID: NDnou
# Question 1: What is the most popular attraction to visit in the park?
# Notes: Your output should be the name of the attraction.
import sqlite3

conn = sqlite3.connect('./readonly/dinofunworld.db')
cursor = conn.cursor()
cursor.execute("""
    SELECT Name, MAX(checkins) 
    FROM (
        SELECT attraction as popular_attraction, COUNT(*) as checkins
        FROM checkin
        GROUP BY popular_attraction
        ORDER BY checkins desc
    ) JOIN attraction
    ON attraction.AttractionID = popular_attraction
    LIMIT 1;
    """)
name_of_popular_attraction = cursor.fetchone()[0]
print(name_of_popular_attraction)

Atmosfear


In [2]:
# Graded Cell, PartID: FXGHp
# Question 2: What ride (note that not all attractions are rides) has the longest average visit time?
# Notes: Your output should be the name of the ride.
import sqlite3

conn = sqlite3.connect('./readonly/dinofunworld.db')
cursor = conn.cursor()
cursor.execute("""
    SELECT AttractionID, AVG(duration) as avg_duration, Name
    FROM checkin
    JOIN (
        SELECT DISTINCT(AttractionID), Name
        FROM attraction
        WHERE Category LIKE '%Ride%'
    )
    ON checkin.attraction = AttractionID
    GROUP BY AttractionID
    ORDER BY avg_duration desc
    LIMIT 1;
    """)
long_average_timed_attraction = cursor.fetchone()[2]
print(long_average_timed_attraction)

Flight of the Swingodon


In [3]:
# Graded Cell, PartID: KALua
# Question 3: Which Fast Food offering in the park has the fewest visitors?
# Notes: Your output should be the name of the fast food offering.
import sqlite3

conn = sqlite3.connect('./readonly/dinofunworld.db')
cursor = conn.cursor()
cursor.execute("""
    SELECT attraction, COUNT(*) as visitor_count, AttractionName
    FROM checkin
    JOIN (
        SELECT AttractionID, Name as AttractionName
        FROM attraction
        WHERE type LIKE '%Fast Food%'
    )
    ON attraction = AttractionID
    GROUP BY attraction
    ORDER BY visitor_count asc
    LIMIT 1;
    """)
fewest_visited_fast_food_joint = cursor.fetchone()[2]
print(fewest_visited_fast_food_joint)

Theresaur Food Stop


In [4]:
# Graded Cell, PartID: B0LUP
# Question 4: Compute the Skyline of number of visits and visit time for the park's ride and 
#  report the rides that appear in the Skyline. 
# Notes: Remember that in this case, higher visits is better and lower visit times are better. 
#  Your output should be formatted as an array listing the names of the rides in the Skyline.
import sqlite3
import math

conn = sqlite3.connect('./readonly/dinofunworld.db')
cursor = conn.cursor()
cursor.execute("""
    SELECT AttractionID, COUNT(*) as visits, AVG(duration)*-10000 as visit_time, Name
    FROM checkin
    JOIN (
        SELECT DISTINCT(AttractionID), Name
        FROM attraction
        WHERE Category LIKE '%Ride%'
    )
    ON checkin.attraction = AttractionID
    GROUP BY AttractionID
    ORDER BY visits asc;
    """)
results = cursor.fetchall()
filtered_bad_durations = list(filter(lambda row: int(row[2]) != 0, results))

# %matplotlib inline
# import matplotlib.pyplot as plt

# x = [row[1] for row in filtered_bad_durations]
# y = [row[2] for row in filtered_bad_durations]
# names = [row[3] for row in filtered_bad_durations]
# fig, ax = plt.subplots()
# fig.set_size_inches(18.5, 30.5)
# ax.scatter(x, y)

# for i, txt in enumerate(names):
#     ax.annotate(txt, (x[i], y[i]))
# plt.show()

bin_a = max(filter(lambda row: int(row[1]) < 10000, filtered_bad_durations), key=lambda row: row[2])
bin_b = max(filter(lambda row: int(row[1]) < 20000, filtered_bad_durations), key=lambda row: row[2])
bin_c = max(filter(lambda row: int(row[1]) > 20000, filtered_bad_durations), key=lambda row: row[2])

print([bin_a[3], bin_b[3], bin_c[3]])


['Scholtz Express', 'Galactosaurus Rage', 'Atmosfear']
